In [1]:
from src import processing, utils

import os
import numpy as np
import pandas as pd
from   glob  import glob
from   re    import sub

# Extract NIFTY images from compressed files

## Extract absolute Jacobian images

In [2]:
# Directory containing compressed images
input_dir_abs = 'data/human/registration/jacobians/absolute/smooth/'
input_dir_abs = os.path.join(input_dir_abs, '')

# List of paths to compressed images
input_files_abs = glob(input_dir_abs+'*.gz')

# Extract absolute jacobians
# unzipped_files_abs = processing.gunzip_files(infiles = input_files_abs, 
#                                              keep = True,
#                                              parallel = True,
#                                              nproc = 8)

unzipped_files_abs = glob(input_dir_abs+'*.nii')
unzipped_files_abs[:5]

['data/human/registration/jacobians/absolute/smooth/d8_0096_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/12013_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/21045_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/d8_0673_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/sub-1050792_ses-01_T1w.extracted_fwhm_4vox.nii']

## Extract relative Jacobian images

In [3]:
# Directory containing compressed images
input_dir_rel = 'data/human/registration/jacobians/relative/smooth/'
input_dir_rel = os.path.join(input_dir_rel, '')

# List of paths to compressed images
input_files_rel = glob(input_dir_rel+'*.gz')

# Extract relative jacobians
# unzipped_files_rel = processing.gunzip_files(infiles = input_files_rel, 
#                                              keep = True,
#                                              parallel = True,
#                                              nproc = 8)

unzipped_files_rel = glob(input_dir_rel+'*.nii')
unzipped_files_rel[:5]

['data/human/registration/jacobians/relative/smooth/d8_0773_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/sub-1050249_ses-01_T1w.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/d8_0214_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/21025_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/sub-3940059_ses-01_run-02_T1w.extracted_fwhm_4vox.nii']

---

# Convert NIFTY images to MINC

## Convert absolute Jacobian images

In [4]:
# Directory in which to save the converted images
outdir_abs = 'data/human/registration/jacobians/absolute/smooth_minc/'

# Convert absolute jacobian images to MINC
# imgfiles_abs = processing.convert_images(infiles = unzipped_files_abs,
#                                          input_format = 'nifty',
#                                          output_format = 'minc',
#                                          outdir = outdir_abs,
#                                          keep = True,
#                                          parallel = True,
#                                          nproc = 8)

imgfiles_abs = glob(outdir_abs+'*.mnc')
imgfiles_abs[:5]

['data/human/registration/jacobians/absolute/smooth_minc/sub-1140440_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050100_ses-01_run-02_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/d8_0285_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050192_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/41014_T1.extracted_fwhm_4vox.mnc']

## Convert relative Jacobian images

In [5]:
# Directory in which to save the converted images
outdir_rel = 'data/human/registration/jacobians/relative/smooth_minc/'

# Convert absolute jacobian images to MINC
# imgfiles_rel = processing.convert_images(infiles = unzipped_files_rel,
#                                          input_format = 'nifty',
#                                          output_format = 'minc',
#                                          outdir = outdir_rel,
#                                          keep = True,
#                                          parallel = True,
#                                          nproc = 8)

imgfiles_rel = glob(outdir_rel+'*.mnc')
imgfiles_rel[:5]

['data/human/registration/jacobians/relative/smooth_minc/32014_T1.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/sub-3940197_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/d8_0219_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/d8_0092_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/11034_T1.extracted_fwhm_4vox.mnc']

---

# Specify which cohorts to use

In [6]:
#Create directory to save data subset derivatives
human_basedir = 'data/human/derivatives'
datasets = ['POND', 'SickKids']    
human_basedir = utils.mkdir_from_list(inlist = datasets,
                                      basedir = human_basedir)
human_basedir

'data/human/derivatives/POND_SickKids'

## Filter demographics for cohorts of interest

In [7]:
#Path to the demographics file 
demographics = 'data/human/registration/DBM_input_demo_passedqc.csv'

#Import demographics
df_demographics = pd.read_csv(demographics)

#Filter individuals for data subset
df_demographics = (df_demographics
                   .loc[df_demographics['Dataset'].isin(datasets)]
                   .copy())

#Create a new column for patient image files
df_demographics['file'] = (df_demographics['Extract_ID']
                           .str.replace('.mnc', 
                                        '_fwhm_4vox.mnc', 
                                        regex = True))

#Write out demographics subset to subset directory
demographics = os.path.join(human_basedir, os.path.basename(demographics))
df_demographics.to_csv(demographics, index = False)

## Create symlinks to absolute jacobian images

In [8]:
#Create output directory for absolute jacobian images
imgdir_abs = os.path.join(human_basedir, 'jacobians', 'resolution_0.5', 'absolute', '')

#Get image file names
imgfiles_abs_dataset = df_demographics['file'].to_list()

#Get image paths for images in the subset
imgfiles_abs_dataset = [[f for f in imgfiles_abs if g in f][0] 
                        for g in imgfiles_abs_dataset]


#Create symbolic links for subset images to the subset directory
imgfiles_abs = utils.mk_symlinks(src = imgfiles_abs_dataset,
                        dst = outdir_abs)
imgfiles_abs[:5]

['data/human/registration/jacobians/absolute/smooth_minc/sub-1050004_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050027_ses-01_run-03_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050032_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050039_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050040_ses-01_T1w.extracted_fwhm_4vox.mnc']

## Create symlinks to relative jacobian images

In [9]:
#Create output directory for absolute jacobian images
imgdir_rel = os.path.join(human_basedir, 'jacobians', 'resolution_0.5', 'relative', '')

#Get image file names
imgfiles_rel_dataset = df_demographics['file'].to_list()

#Get image paths for images in the subset
imgfiles_rel_dataset = [[f for f in imgfiles_rel if g in f][0] 
                        for g in imgfiles_rel_dataset]

#Create symbolic links for subset images to the subset directory
imgfiles_rel = utils.mk_symlinks(src = imgfiles_rel_dataset,
                                 dst = imgdir_rel)
imgfiles_rel[:5]

['data/human/derivatives/POND_SickKids/jacobians/resolution_0.5/relative/sub-1050004_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_0.5/relative/sub-1050027_ses-01_run-03_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_0.5/relative/sub-1050032_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_0.5/relative/sub-1050039_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_0.5/relative/sub-1050040_ses-01_T1w.extracted_fwhm_4vox.mnc']

---

# Downsample jacobian images

## Downsampling absolute jacobian images to 3.0mm

In [10]:
# Resolution (mm) to which we want to downsample
isostep = 3.0

In [11]:
# Create a separate output directory for downsampled images
imgdir_abs_3mm = sub(r'resolution_0.5',
                    'resolution_{}'.format(isostep),
                    imgdir_abs)

# imgfiles_abs_3mm = processing.resample_images(infiles = imgfiles_abs,
#                                               isostep = isostep,
#                                               outdir = imgdir_abs_3mm,
#                                               parallel = True,
#                                               nproc = 8)

imgfiles_abs_3mm = glob(imgdir_abs_3mm+'*.mnc')
imgfiles_abs_3mm[:5]

['data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/absolute/d8_0376_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/absolute/d8_0141_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/absolute/d8_0048_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/absolute/d8_0080_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/absolute/d8_0665_01.extracted_fwhm_4vox.mnc']

## Downsampling relative jacobian images to 3.0mm

In [12]:
# Create a separate output directory for downsampled images
imgdir_rel_3mm = sub(r'resolution_0.5',
                    'resolution_{}'.format(isostep),
                    imgdir_rel)

# imgfiles_rel_3mm = processing.resample_images(infiles = imgfiles_rel,
#                                               isostep = isostep,
#                                               outdir = imgdir_rel_3mm,
#                                               parallel = True,
#                                               nproc = 12)

imgfiles_rel_3mm = glob(imgdir_rel_3mm+'*.mnc')
imgfiles_rel_3mm[:5]

['data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/relative/sub-3940216_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/relative/sub-1140430_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/relative/sub-0881063_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/relative/sub-1050698_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/resolution_3.0/relative/sub-0880494_ses-01_T1w.extracted_fwhm_4vox.mnc']

# Downsampling auxiliary imaging files

In [13]:
# Path to the neuroanatomical template image
model = 'data/human/registration/reference_files/model.mnc'

# Path to the mask image
mask = 'data/human/registration/reference_files/mask.mnc'

# # Downsample the template to 3.0 mm
# model_3mm = processing.resample_image(infile = model,
#                                       isostep = 3.0,
#                                       suffix = '_3.0mm')
model_3mm = 'data/human/registration/reference_files/model_3.0mm.mnc'

# # Downsample the mask to 3.0 mm
# mask_3mm = processing.resample_image(infile = mask,
#                                      isostep = 3.0,
#                                      suffix = '_3.0mm')
mask_3mm = 'data/human/registration/reference_files/mask_3.0mm.mnc'

---

# Calculating human participant effect sizes

In [14]:
#Parameters for effect size calculation
method = 'normative-growth'
df = 5
combat = True
combat_batch = ['Site', 'Scanner']
ncontrols = None

## Absolute jacobian effect sizes

In [15]:
es_basedir_abs = os.path.join(human_basedir,
                              'effect_sizes',
                              'resolution_3.0',
                              'absolute',
                              '')


es_params = {'es_method':method,
            'es_df':df,
            'es_combat':combat,
            'es_combat_batch':'-'.join(combat_batch),
            'es_ncontrols':ncontrols}

(es_dir_abs, es_metadata) = utils.mkdir_from_params(params = es_params,
                                                    basedir = es_basedir_abs,
                                                    return_metadata = True)

In [16]:
es_files_abs = processing.calculate_human_effect_sizes(imgdir = imgdir_abs_3mm,
                                                       demographics = demographics,
                                                       mask = mask_3mm,
                                                       outdir = es_dir_abs,
                                                       method = method,
                                                       df = df,
                                                       combat = combat,
                                                       combat_batch = combat_batch,
                                                       parallel = True,
                                                       nproc = 8)
es_files_abs[:5]

Building voxel matrix...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 849/849 [00:09<00:00, 93.87it/s]


Writing out voxel matrix...
Executing normative growth modelling...


Warning message:
no DISPLAY variable so Tk is not available 
Normalizing using batch variable: Site
Found2batches
Adjusting for0covariate(s) or covariate level(s)


Standardizing Data across genes


...

Standardizing Data across genes


Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data



  |======================================================================| 100%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [00:13<00:00, 32.03it/s]


['data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/absolute/106/d8_0001_04.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/absolute/106/d8_0002_03.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/absolute/106/d8_0003_02.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/absolute/106/d8_0004_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/absolute/106/d8_0006_03.extracted_fwhm_4vox.mnc']

## Relative jacobian effect sizes

In [17]:
df_metadata = pd.read_csv(es_metadata, dtype = 'str')
df_metadata

,es_method,es_df,es_combat,es_combat_batch,es_ncontrols,id
0,normative-growth,5,True,Site-Scanner,NaN,106


In [18]:
es_params_id = pd.merge(df_metadata, 
                        pd.DataFrame(es_params, index = [0], 
                                     dtype = 'str'), 
                        how = 'inner').id[0]

es_basedir_rel = os.path.join(human_basedir,
                              'effect_sizes',
                              'resolution_3.0',
                              'relative',
                              '')

es_dir_rel = utils.mkdir_from_params(params = es_params,
                                     basedir = es_basedir_rel, 
                                     params_id=es_params_id,
                                     return_metadata = False)

In [19]:
es_files_rel = processing.calculate_human_effect_sizes(imgdir = imgdir_rel_3mm,
                                                       demographics = demographics,
                                                       mask = mask_3mm,
                                                       outdir = es_dir_rel,
                                                       method = method,
                                                       df = df,
                                                       combat = combat,
                                                       combat_batch = combat_batch,
                                                       parallel = True,
                                                       nproc = 8)

es_files_rel[:5]

Building voxel matrix...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 849/849 [00:09<00:00, 92.38it/s]


Writing out voxel matrix...
Executing normative growth modelling...


Warning message:
no DISPLAY variable so Tk is not available 
Normalizing using batch variable: Site
Found2batches
Adjusting for0covariate(s) or covariate level(s)


Standardizing Data across genes


...

Standardizing Data across genes


Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data



  |======================================================================| 100%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [00:14<00:00, 28.45it/s]


['data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/relative/106/d8_0001_04.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/relative/106/d8_0002_03.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/relative/106/d8_0003_02.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/relative/106/d8_0004_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/resolution_3.0/relative/106/d8_0006_03.extracted_fwhm_4vox.mnc']

---

# Clustering the participants

## Creating effect size matrices

### Absolute effect size matrix

In [22]:
# File in which to save the absolute effect size matrix
es_abs_matrix_csv = 'effect_sizes.csv'
es_abs_matrix_csv = os.path.join(es_dir_abs, es_abs_matrix_csv)

# Compute the absolute effect size matrix
df_es_abs = processing.build_voxel_matrix(infiles = es_files_abs,
                                          mask = mask_3mm,
                                          sort = True,
                                          file_col = True,
                                          parallel = False,
                                          nproc = 2)

# Extract file base names from paths
df_es_abs['file'] = [os.path.basename(file) for file in df_es_abs['file']]

# Write to file
df_es_abs.to_csv(es_abs_matrix_csv, index = False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [01:21<00:00,  5.19it/s]


### Relative effect size matrix

In [24]:
# File in which to save the absolute effect size matrix
es_rel_matrix_csv = 'effect_sizes.csv'
es_rel_matrix_csv = os.path.join(es_dir_rel, es_rel_matrix_csv)

# Compute the absolute effect size matrix
df_es_rel = processing.build_voxel_matrix(infiles = es_files_rel,
                                          mask = mask_3mm,
                                          sort = True,
                                          file_col = True,
                                          parallel = False,
                                          nproc = 2)

# Extract file base names from paths
df_es_rel['file'] = [os.path.basename(file) for file in df_es_rel['file']]

# Write to file
df_es_rel.to_csv(es_rel_matrix_csv, index = False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [01:07<00:00,  6.31it/s]


# Clustering the participants

In [59]:
#Cluster parameters
infiles = [es_abs_matrix_csv, es_rel_matrix_csv]
nk_max = 10
metric = 'correlation'
K = 10
sigma = 0.5
t = 20

#Combine cluster params with previous params
cluster_params = es_params.copy()
cluster_params.update({'clust_nk_max':nk_max,
                       'clust_metric':metric,
                       'clust_K':K,
                       'clust_sigma':sigma,
                       'clust_t':t})

#Create cluster params ID based on previous ID
cluster_params_id = '-'.join([es_params_id, utils.random_id(3)])

#Create cluster params directory
cluster_basedir = os.path.join(human_basedir, 'clusters')
(cluster_dir, cluster_metadata) = utils.mkdir_from_params(params = cluster_params,
                                                            basedir = cluster_basedir,
                                                          params_id = cluster_params_id,
                                                            return_metadata = True)

cluster_file = os.path.join(cluster_dir, 'human_clusters.csv')

data/human/derivatives/POND_SickKids/clusters/106-222/


In [60]:
df_clusters = processing.cluster_human_data(infiles = infiles,
                                            rownames = 'file',
                                            nk_max = nk_max,
                                            metric = metric,
                                            K = K,
                                            sigma = sigma,
                                            t = t,
                                            cluster_file = cluster_file)

df_clusters.head()

Importing data...
Running similarity network fusion...
Assigning clusters...


,ID,nk2,nk3,nk4,nk5,nk6,nk7,nk8,nk9,nk10
0,d8_0001_04.extracted_fwhm_4vox.mnc,1,1,1,1,1,1,1,1,1
1,d8_0002_03.extracted_fwhm_4vox.mnc,1,2,3,2,2,6,3,6,8
2,d8_0003_02.extracted_fwhm_4vox.mnc,1,1,1,1,6,1,8,8,9
3,d8_0004_01.extracted_fwhm_4vox.mnc,2,3,4,3,4,3,7,9,3
4,d8_0006_03.extracted_fwhm_4vox.mnc,2,3,4,3,4,3,7,9,3


In [61]:
df_cluster_metadata = pd.read_csv(cluster_metadata)
df_cluster_metadata

,es_method,es_df,es_combat,es_combat_batch,es_ncontrols,clust_nk_max,clust_metric,clust_K,clust_sigma,clust_t,id
0,normative-growth,5,True,Site-Scanner,NaN,10,correlation,10,0.5,20,106-222


**To-do**: Implement W-matrix in clustering

# Create cluster maps

## Create absolute cluster maps

In [46]:
df_cluster_metadata  = pd.read_csv(cluster_metadata)
df_cluster_params = pd.DataFrame(cluster_params, index = [0])
df_cluster_params = pd.merge(df_cluster_metadata, df_cluster_params, how = 'inner')
cluster_id = str(df_cluster_params['id'].values[0])

cluster_map_params = cluster_params.copy()
cluster_map_params['clust_map_method'] = 'mean'

cluster_map_id = '-'.join([cluster_id, utils.random_id(3)])

cluster_map_basedir = os.path.join(human_basedir, 'cluster_maps')
cluster_map_basedir_abs = os.path.join(cluster_map_basedir, 'absolute')

(cluster_map_dir_abs, metadata_cluster_map) = utils.mkdir_from_params(params = cluster_map_params, 
                                                                      basedir = cluster_map_basedir_abs,
                                                                      params_id = cluster_map_id,
                                                                      return_metadata = True)
cluster_map_dir_abs

'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/'

In [39]:
# cluster_maps_abs = processing.create_cluster_maps(clusters = cluster_file,
#                                                   imgdir = es_dir_abs_1mm,
#                                                   outdir = cluster_map_dir_abs,
#                                                   method = 'mean',
#                                                   mask = mask_1mm)

cluster_maps_abs = glob(cluster_map_dir_abs+'*.mnc')
cluster_maps_abs[:5]

['data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_9_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_4_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_5_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_10_k_8_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_10_k_7_mean.mnc']

## Create relative cluster maps

In [55]:
df_cluster_map_metadata  = pd.read_csv(metadata_cluster_map)
df_cluster_map_params = pd.DataFrame(cluster_map_params, index = [0])
df_cluster_map_params = pd.merge(df_cluster_map_metadata, df_cluster_map_params, how = 'inner')
cluster_id = str(df_cluster_map_params['id'].values[0])

cluster_map_basedir_rel = os.path.join(cluster_map_basedir, 'relative')

cluster_map_dir_rel = utils.mkdir_from_params(params = cluster_map_params,
                                              basedir = cluster_map_basedir_rel,
                                              params_id = cluster_id)

In [56]:
cluster_maps_rel = processing.create_cluster_maps(clusters = cluster_file,
                                                  imgdir = es_dir_rel_1mm,
                                                  outdir = cluster_map_dir_rel,
                                                  method = 'mean',
                                                  mask = mask_1mm)

cluster_maps_rel = glob(cluster_map_dir_rel+'*.mnc')
cluster_maps_rel[:5]

Importing cluster information...
Creating cluster maps...
nk = 2 : k = 1
nk = 2 : k = 2
nk = 3 : k = 1
nk = 3 : k = 2
nk = 3 : k = 3
nk = 4 : k = 1
nk = 4 : k = 2
nk = 4 : k = 3
nk = 4 : k = 4
nk = 5 : k = 1
nk = 5 : k = 2
nk = 5 : k = 3
nk = 5 : k = 4
nk = 5 : k = 5
nk = 6 : k = 1
nk = 6 : k = 2
nk = 6 : k = 3
nk = 6 : k = 4
nk = 6 : k = 5
nk = 6 : k = 6
nk = 7 : k = 1
nk = 7 : k = 2
nk = 7 : k = 3
nk = 7 : k = 4
nk = 7 : k = 5
nk = 7 : k = 6
nk = 7 : k = 7
nk = 8 : k = 1
nk = 8 : k = 2
nk = 8 : k = 3
nk = 8 : k = 4
nk = 8 : k = 5
nk = 8 : k = 6
nk = 8 : k = 7
nk = 8 : k = 8
nk = 9 : k = 1
nk = 9 : k = 2
nk = 9 : k = 3
nk = 9 : k = 4
nk = 9 : k = 5
nk = 9 : k = 6
nk = 9 : k = 7
nk = 9 : k = 8
nk = 9 : k = 9
nk = 10 : k = 1
nk = 10 : k = 2
nk = 10 : k = 3
nk = 10 : k = 4
nk = 10 : k = 5
nk = 10 : k = 6
nk = 10 : k = 7
nk = 10 : k = 8
nk = 10 : k = 9
nk = 10 : k = 10


['data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_7_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_10_k_5_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_6_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_9_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_4_k_2_mean.mnc']

In [24]:
# cluster_metadata = os.path.join(clustering_basedir, 'metadata.csv')
# df_cluster_metadata = pd.read_csv(cluster_metadata)
# df_cluster_metadata = df_cluster_metadata[(df_cluster_metadata['nk_max'] == nk_max) & 
#                                           (df_cluster_metadata['metric'] == metric) & 
#                                           (df_cluster_metadata['K'] == K) &
#                                           (df_cluster_metadata['sigma'] == sigma) & 
#                                           (df_cluster_metadata['t'] == t)] 

---